In [ ]:
!pip -q install torch torchvision torchaudio transformers datasets scikit-learn


In [ ]:
import pandas as pd

DATA_FILE = "kai_labeled_final_test_sementara.csv"
df = pd.read_csv(DATA_FILE)

print("Distribusi Label:")
print(df["LABEL"].value_counts())
df.head(10)


Distribusi Label:
LABEL
neu    862
neg    364
pos    208
Name: count, dtype: int64


,id_str,created_at,user_id_str,conversation_id_str,text_clean,LABEL
0,1608764220451213313,Fri Dec 30 09:57:34 +0000 2022,230609460,1608764220451213313,udah beli tiket di aplikasi kaiaccess dan saat...,neu
1,1608631716184489986,Fri Dec 30 01:11:02 +0000 2022,1158718283870138368,1608631716184489986,kenapa gabisa pake ovo buat pembayaran kaiacce...,neg
2,1608263340564058116,Thu Dec 29 00:47:15 +0000 2022,1250731212471074816,1608263340564058116,min saya mau ubah no hp yang ada di kaiaccess ...,neu
3,1608235238366535680,Wed Dec 28 22:55:35 +0000 2022,120352418,1608227080503955458,waktunya menikmati pemandangan sepanjang perja...,pos
4,1608049638111850496,Wed Dec 28 10:38:04 +0000 2022,967360502652223489,1608049638111850496,kenapa kaiaccess dari kemaren susah di acces l...,neg
5,1607954768332492802,Wed Dec 28 04:21:05 +0000 2022,1010877687932440577,1607954768332492802,halo saya abis beli tiket kereta di aplikasi k...,neg
6,1607943108934914049,Wed Dec 28 03:34:45 +0000 2022,71733078,1607943108934914049,min aplikasi kaiaccess lagi ngadat kah berat b...,neg
7,1607879185586061312,Tue Dec 27 23:20:45 +0000 2022,438072886,1607879185586061312,ada yang tau jam buka loket stasiun sidoarjo b...,pos
8,1607755461062950912,Tue Dec 27 15:09:07 +0000 2022,414602951,1607755461062950912,halo apps nya lagi under constructions apa jar...,neu
9,1607603636284325890,Tue Dec 27 05:05:49 +0000 2022,359810843,1607603636284325890,hallo kenapaa app kaiaccess tidak bisa digunak...,neg


In [ ]:
from sklearn.model_selection import train_test_split

X = df["text_clean"].astype(str).tolist()  # pilih kolom teks yang mau dipakai
y = df["LABEL"].astype(str).tolist()

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Train size:", len(X_train))
print("Test size:", len(X_test))


Train size: 1147
Test size: 287


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModel

MODEL_NAME = "indolem/indobertweet-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME)

def get_embeddings(texts, tokenizer, model, max_len=128):
    tokens = tokenizer(
        texts,
        max_length=max_len,
        truncation=True,
        padding=True,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = model(**tokens)
    return outputs.last_hidden_state[:,0,:].numpy()  # CLS token

train_embeds = get_embeddings(X_train, tokenizer, model)
test_embeds = get_embeddings(X_test, tokenizer, model)

train_embeds.shape, test_embeds.shape

tokenizer_config.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/445M [00:00<?, ?B/s]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

params = {
    "n_estimators": [100, 200],
    "max_depth": [None, 10, 20]
}

rf = RandomForestClassifier(random_state=42, class_weight="balanced")
grid = GridSearchCV(rf, params, cv=3, scoring="f1_macro", n_jobs=-1)
grid.fit(train_embeds, y_train)

print("Best Params:", grid.best_params_)

y_pred = grid.best_estimator_.predict(test_embeds)
print(classification_report(y_test, y_pred, digits=4))